# Agente de IA Multimodal para Extração de dados de Documentos

## Visão Geral do Projeto

Este projeto implementa um **Agente de IA Multimodal** capaz de analisar e extrair informações de documentos, para exemplificação serão utilizados notas fiscais eletrônicas (NFS-e). O agente utiliza duas abordagens complementares:

1. **Análise por OCR (Optical Character Recognition)**: Extrai texto de PDFs escaneados usando OCR
2. **Análise Direta de Arquivos PDF**: Envia o arquivo PDF diretamente para o agente de IA analisar


- **Modelo de IA**: No projeto será utilizado o modelo Google Gemini 2.5 Flash ou 2.0 Flash Lite
- **Framework de Agentes**: Utiliza o PydanticAI para criar e gerenciar agentes de IA

### Dados Extraídos
O agente extrai os seguintes campos de notas fiscais:
- Descrição do serviço
- Valor do serviço
- Número da nota fiscal
- Data de emissão
- Valor total
- CNPJ do prestador


In [1]:
# Importações necessárias
from utils.Agente import AI_agente_extrator
from utils.pdf_to_img_to_text import extrair_texto_de_pdf_scaneado

from pathlib import Path


### 1. Inicialização do Agente de IA
Criar uma instância do agente de IA que será responsável por analisar e extrair informações dos documentos.

In [2]:
agent = AI_agente_extrator()

Modelo gemini-2.5-flash criado com sucesso.


In [3]:
agent.agent

Agent(model=GoogleModel(), name=None, end_strategy='early', model_settings=None, output_type=<class 'utils.BaseModel.ExtracaoOutput'>, instrument=None)

### 2. Carregamento de Arquivos PDF
### Objetivo
Localizar e contar todos os arquivos PDF na pasta de notas fiscais que serão processados.
### Processo
1. Verifica se a pasta 'nota_fiscal' existe
2. Busca todos os arquivos com extensão .pdf na pasta
3. Exibe a quantidade de arquivos encontrados



In [4]:
# Verificação da pasta de notas fiscais
pasta_notas = Path('nota_fiscal')
if not pasta_notas.exists():
    print(f"Pasta '{pasta_notas}' não encontrada.")

# Busca todos os arquivos PDF na pasta
arquivos_pdf = list(pasta_notas.glob('*.pdf'))
if not arquivos_pdf:
    print("Nenhum arquivo PDF encontrado na pasta 'nota_fiscal'.")

print(f"Encontrados {len(arquivos_pdf)} arquivos PDF para processar...")

Encontrados 1 arquivos PDF para processar...


### 3. Extração de Texto via OCR (Abordagem 1)

### Objetivo
Extrair texto do PDF escaneado usando Optical Character Recognition (OCR).

### Processo
1. Converte cada página do PDF em imagem
2. Aplica OCR para extrair o texto da imagem
3. Retorna o texto completo do documento

In [5]:
arquivo_txt = extrair_texto_de_pdf_scaneado(arquivos_pdf[0])
arquivo_txt

PDF tem 1 página(s). Processando...
Processando página 1...
Processamento concluído.


"--- PÁGINA 1 ---\nRPS Nº 4552062 Série RPS, emitido em 30/09/2025| Numero da Nota\nPREFEITURA DO 04365344\n\nLemes Data e Hora de Emissão\nR E C I F E e 02/10/2025 15:12:19\nSECRETARIA DE FINANÇAS Código de Verificação\n\nNota Fiscal de Serviços Eletrônica IC44-CPYF\nPRESTADOR DE SERVIÇOS\nCPF/CNPJ: 04.986.320/0031-39 Inscrição Municipal: 511.256-7\nNome/Razão Social: SER EDUCACIONAL S.A.\nEndereço: AVENIDA GUARARAPES 283, LOJA 3 - SANTO ANTONIO - CEP: 50010-000\nMunicípio: Recife UF: PE E-mail: contabilidade()sereducacional.com\n\nTOMADOR DE SERVIÇOS\nNome/Razão Social: ALEX SILVA DE ASSUNÇÃO\n\ncrrCNPJDD O Inscrição Municipal: --—-\nEndereçETTTTTI\n\nMunicípio: Curitiba ur: PR E-mail: alexsaweb 6) gmail.com\n\nDISCRIMINAÇÃO DOS SERVIÇOS\n\nMENSALIDADE EAD\n\nVALOR TOTAL DO SERVIÇO = R$ 138,33\n\nCódigo da Atividade Prestada a . .\n8532500 - EDUCAÇÃO SUPERIOR - GRADUAÇÃO E POS-GRADUAÇÃO\n\n08.01 - Ensino regular pré escolar, fundamental, médio e superior.\n\nDeduções (R$) Desconto In

### 4. Análise de Texto pelo Agente (Abordagem 1)

### Objetivo
Enviar o texto extraído via OCR para o agente de IA analisar e extrair os dados estruturados.

### Processo
1. O agente recebe o texto bruto do PDF
2. Utiliza o modelo Gemini para análise
3. Extrai os campos específicos da nota fiscal
4. Retorna os dados em formato JSON estruturado

In [6]:
resultado_txt = await agent.extrair_dados_txt(arquivo_txt)

In [7]:
resultado_txt

'{"descricao_servico":"MENSALIDADE EAD","valor_servico":138.33,"numero_nf":4365344,"data_emissao":"2025-10-02","valor_total":138.33,"cnpj":"04986320003139"}'

### 5. Carregamento do PDF em Bytes (Abordagem 2)

### Objetivo
Carregar o arquivo PDF como bytes para envio direto ao agente de IA, sem necessidade de OCR prévio.

### Processo
1. Especifica o caminho do arquivo PDF
2. Lê o arquivo como bytes (dados binários)
3. Prepara os dados para envio ao agente

In [ ]:
pdf_nota = 'nota_fiscal/NFSe.pdf'
arquivobyte = Path(pdf_nota).read_bytes()

### 6. Análise de PDF pelo Agente (Abordagem 2)

### Objetivo
Enviar o arquivo PDF diretamente para o agente de IA analisar, sem OCR prévio.

### Processo
1. O agente recebe o arquivo PDF
2. Utiliza BinaryContent para lidar com o arquivo binário 
3. Utiliza o modelo Gemini para extrair os dados
4. Retorna os dados em formato JSON estruturado


In [ ]:
resultado_pdf = await agent.extrair_dados_pdf(arquivobyte)

In [10]:
resultado_pdf

'{"descricao_servico":"MENSALIDADE EAD","valor_servico":138.33,"numero_nf":4365344,"data_emissao":"2025-10-02","valor_total":138.33,"cnpj":"04986320003139"}'